Lesson 11
# OpenAIツール呼出：Web検索などとの連動

Chapter 5

In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '東京駅周辺のイベント - 駅探 駅探 | ekitan *   乗換案内 *   飛行機時刻表 *   飛行機時刻表(国内線) *   飛行機時刻表(国際線)NEW *   高速・夜行バス時刻表 NEW 1.   駅探 3.   東京駅周辺のイベント *   条件 - [x] 無料イベント  - [x] 駐車場あり  - [x] 屋内    *   Image 3: 環境省主催 教職員等環境教育・学習推進リーダー養成研修 実践編 対面研修 東京会場 #### 環境省主催 教職員等環境教育・学習推進リーダー養成研修 実践編 対面研修 東京会場 学校教育×SDGs、環境教育・ESD実践 期間 2025年8月18日(月) 会場 ビジョンセンター東京駅前 * 東京都 中央区 * 東京駅／日本橋駅(東京)／京橋駅(東京)駅 1. *   Image 8: 朝ガツ 東京10社巡り開運ウォーキング-城南4社で運気アップ- #### 朝ガツ 東京10社巡り開運ウォーキング-城南4社で運気アップ- 東京十社巡りウォーキングで開運祈願 期間 2025年5月17日(土)このイベントは終了しました 会場 東京駅丸の内駅前広場（集合） * 東京都 千代田区 * 東京駅／二重橋前駅／大手町駅(東京)駅 *   Image 12 東京メトロ日比谷線 八丁堀駅 徒歩4分 36.6万円(管理費等：14,000円) 2LDK / 74.9m 2 / 12階 / 築18年 Image 13 東京メトロ有楽町線 銀座一丁目駅 徒歩4分 22.5万円(管理費等：13,000円) 1LDK / 49.34m 2 / 13階 / 築21年 Image 14 JR山手線 神田駅 徒歩3分 20.0万円(管理費等：20,000円) 1LDK / 40.58m 2 / 4階 / 築1年 東京駅周辺の物件一覧を見る   Copyright©2025 駅探.',
   'score': 0.7938904,
   'raw_content': None},
  {'url': 'https://www.walk

In [5]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })]
    

In [6]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [7]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [8]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [9]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。具体的には：

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県の方が東京都よりも約77平方キロメートル広いことになります。


In [10]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内の東京駅周辺のイベント情報を以下にまとめました。

1. **T3 PHOTO FESTIVAL TOKYO 2022**  
   - **日時**: 10月1日（土）より開催  
   - **内容**: 鉄道に関する写真展などが実施されます。  
   - **詳細**: [T3 PHOTO FESTIVAL詳細](https://guidetokyo.info/join/report/index.html)

2. **東京駅周辺のイベント情報**  
   - **日時**: 随時開催中の各種イベント  
   - **内容**: 東京駅周辺では多くのイベントが行われ、特に8月には様々な屋外イベントや飲食イベントが予定されています。  
   - **詳細**: [WalkerPlus イベント一覧](https://www.walkerplus.com/event_list/ar0313/sc309880d/)

3. **東京都内のおすすめイベント**  
   - **期間**: 2025年のイベント予定（例）  
   - **内容**: ジャズや多国籍の料理フェスティバル等、多彩なイベントが企画されています。  
   - **詳細**: [Enjoy Tokyo イベント情報](https://www.enjoytokyo.jp/event/list/ranking/its05/)

4. **あめりか村祭り**  
   - **日時**: 11月9日（木）に開催予定  
   - **内容**: 地元の文化を感じるための祭り。詳細情報は随時更新されます。  
   - **詳細**: [あめりか村祭りの詳細](https://note.com/beyond_ama_fes/n/ne6aa4513738c)

これらの情報を参考に、ぜひ東京駅周辺のイベントを楽しんでください！


In [11]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:こんにちは!'

こんにちは！何かお手伝いできることはありますか？


'質問:東北6県は？'

東北地方は、日本の北東部に位置する地域で、以下の6つの県があります。

1. 青森県 (あおもりけん)
2. 岩手県 (いわてけん)
3. 宮城県 (みやぎけん)
4. 秋田県 (あきたけん)
5. 山形県 (やまがたけん)
6. 福島県 (ふくしまけん)

これらの県は自然豊かで、歴史や文化も多様です。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産には以下のようなものがあります。

1. **牛タン** - 宮城県名物の牛タンは、観光客に非常に人気があります。多くの店でスライスされたものや、加工品が販売されています。
   - 詳細: [楽天トラベル](https://travel.rakuten.co.jp/mytrip/howto/sendai-souvenir)

2. **ずんだ餅** - 枝豆を使った餡を載せた餅で、甘さと香りが癖になります。
   - 詳細: [サキドリ](https://sakidori.co/article/1320827)

3. **笹かまぼこ** - 魚のすり身を使ったお土産で、食感が特徴です。焼きたてを楽しむこともできます。
   - 詳細: [Atlast](https://atlas-log.com/miyagi-souvenir/)

4. **仙台味噌** - 仙台の地味噌で、料理に深みを与えてくれる調味料です。
   - 詳細: [トラベル](https://travel.rakuten.co.jp/mytrip/howto/sendai-souvenir)

5. **萩の月** - 人気のある洋菓子で、カスタードクリームを包んだお菓子です。
   - 詳細: [楽天トラベル](https://travel.rakuten.co.jp/mytrip/howto/sendai-souvenir)

6. **ままどおる** - もちっとした食感のお菓子で、あんこが入っています。
   - 詳細: [センス](https://tohokuru.jp/blogs/feature/miyage_miyagi?srsltid=AfmBOoq5H79fJMXDppH8HuieTp3qRuRi7fjPqJ6Kw5XBJJMMk4NtMlL8)

これらは特にお勧めの品で、宮城県を訪れた際にはぜひお土産として購入してみてください。

---ご利用ありがとうございました！---
